In [ ]:
!pip install pyiqa

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 26.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 17.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.2/276.2 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 78.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 333.2/333.2 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.4/299.4 kB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.4/226.4 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Cell 1: Environment Setup & Drive Mounting (Updated)
# ====================================================
from google.colab import drive
import os
import sys
import subprocess

# 1. Mount Google Drive
drive.mount('/content/drive')

# 2. Define the project root directory
PROJECT_ROOT = "/content/drive/My Drive/ECE253_Project"

# 3. Define sub-directories
DIRS = {
    "GT": os.path.join(PROJECT_ROOT, "1_GroundTruth"),      # Folder for Sharp images
    "BLUR": os.path.join(PROJECT_ROOT, "2_InputBlur"),      # Folder for Blurred images
    "RL_OUT": os.path.join(PROJECT_ROOT, "3_Output_RL"),    # Folder for RL results
    "NAF_OUT": os.path.join(PROJECT_ROOT, "4_Output_NAFNet"), # Folder for Pre-trained NAFNet results
    "NAF_FT_OUT": os.path.join(PROJECT_ROOT, "5_Output_NAFNet_FineTuned"),

    "METRICS": os.path.join(PROJECT_ROOT, "Metrics")        # Folder for CSV files
}

# 4. Create directories if they don't exist
for k, v in DIRS.items():
    os.makedirs(v, exist_ok=True)

print(f"✅ Working directory ready: {PROJECT_ROOT}")
print(f"📂 Created dedicated Fine-Tune output folder: {DIRS['NAF_FT_OUT']}")

# 5. Install necessary libraries
print(">>> Installing dependencies...")
subprocess.check_call([sys.executable, "-m", "pip", "install", "datasets", "scikit-image", "tqdm", "pandas", "opencv-python"])
print("✅ Dependencies installed.")

Mounted at /content/drive
✅ Working directory ready: /content/drive/My Drive/ECE253_Project
📂 Created dedicated Fine-Tune output folder: /content/drive/My Drive/ECE253_Project/5_Output_NAFNet_FineTuned
>>> Installing dependencies...
✅ Dependencies installed.


In [ ]:
# Cell 2: NAFNet Architecture & Pre-trained Weight Loading
# ========================================================
import torch
import torch.nn as nn
import torch.nn.functional as F
import os

# --- 1. NAFNet Architecture Definitions ---

class LayerNormFunction(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x, weight, bias, eps):
        ctx.eps = eps
        N, C, H, W = x.size()
        mu = x.mean(1, keepdim=True)
        var = (x - mu).pow(2).mean(1, keepdim=True)
        y = (x - mu) / (var + eps).sqrt()
        ctx.save_for_backward(y, var, weight)
        y = weight.view(1, C, 1, 1) * y + bias.view(1, C, 1, 1)
        return y

    @staticmethod
    def backward(ctx, grad_output):
        eps = ctx.eps
        N, C, H, W = grad_output.size()
        y, var, weight = ctx.saved_tensors
        g = grad_output * weight.view(1, C, 1, 1)
        mean_g = g.mean(dim=1, keepdim=True)
        mean_gy = (g * y).mean(dim=1, keepdim=True)
        gx = 1. / torch.sqrt(var + eps) * (g - y * mean_gy - mean_g)
        return gx, (grad_output * y).sum(dim=3).sum(dim=2).sum(dim=0), grad_output.sum(dim=3).sum(dim=2).sum(dim=0), None

class LayerNorm2d(nn.Module):
    def __init__(self, channels, eps=1e-6):
        super(LayerNorm2d, self).__init__()
        self.register_parameter('weight', nn.Parameter(torch.ones(channels)))
        self.register_parameter('bias', nn.Parameter(torch.zeros(channels)))
        self.eps = eps
    def forward(self, x):
        return LayerNormFunction.apply(x, self.weight, self.bias, self.eps)

class SimpleGate(nn.Module):
    def forward(self, x):
        x1, x2 = x.chunk(2, dim=1)
        return x1 * x2

class NAFBlock(nn.Module):
    def __init__(self, c, DW_Expand=2, FFN_Expand=2, drop_out_rate=0.):
        super().__init__()
        dw_channel = c * DW_Expand
        self.conv1 = nn.Conv2d(in_channels=c, out_channels=dw_channel, kernel_size=1, padding=0, stride=1, groups=1, bias=True)
        self.conv2 = nn.Conv2d(in_channels=dw_channel, out_channels=dw_channel, kernel_size=3, padding=1, stride=1, groups=dw_channel, bias=True)
        self.conv3 = nn.Conv2d(in_channels=dw_channel // 2, out_channels=c, kernel_size=1, padding=0, stride=1, groups=1, bias=True)
        self.sca = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(in_channels=dw_channel // 2, out_channels=dw_channel // 2, kernel_size=1, padding=0, stride=1, groups=1, bias=True),
        )
        self.conv4 = nn.Conv2d(in_channels=c, out_channels=c * FFN_Expand, kernel_size=1, padding=0, stride=1, groups=1, bias=True)
        self.conv5 = nn.Conv2d(in_channels=c * FFN_Expand // 2, out_channels=c, kernel_size=1, padding=0, stride=1, groups=1, bias=True)
        self.norm1 = LayerNorm2d(c)
        self.norm2 = LayerNorm2d(c)
        self.dropout1 = nn.Dropout(drop_out_rate) if drop_out_rate > 0. else nn.Identity()
        self.dropout2 = nn.Dropout(drop_out_rate) if drop_out_rate > 0. else nn.Identity()
        self.beta = nn.Parameter(torch.zeros((1, c, 1, 1)), requires_grad=True)
        self.gamma = nn.Parameter(torch.zeros((1, c, 1, 1)), requires_grad=True)

    def forward(self, x):
        inp = x
        x = self.norm1(x)
        x = self.conv1(x)
        x = self.conv2(x)
        x = SimpleGate()(x)
        x = x * self.sca(x)
        x = self.conv3(x)
        x = self.dropout1(x)
        y = inp + x * self.beta
        x = self.norm2(y)
        x = self.conv4(x)
        x = SimpleGate()(x)
        x = self.conv5(x)
        x = self.dropout2(x)
        return y + x * self.gamma

class NAFNet(nn.Module):
    def __init__(self, img_channel=3, width=16, middle_blk_num=1, enc_blk_nums=[], dec_blk_nums=[]):
        super().__init__()
        self.intro = nn.Conv2d(in_channels=img_channel, out_channels=width, kernel_size=3, padding=1, stride=1, groups=1, bias=True)
        self.ending = nn.Conv2d(in_channels=width, out_channels=img_channel, kernel_size=3, padding=1, stride=1, groups=1, bias=True)
        self.encoders = nn.ModuleList()
        self.decoders = nn.ModuleList()
        self.middle_blks = nn.ModuleList()
        self.ups = nn.ModuleList()
        self.downs = nn.ModuleList()
        chan = width
        for num in enc_blk_nums:
            self.encoders.append(nn.Sequential(*[NAFBlock(chan) for _ in range(num)]))
            self.downs.append(nn.Conv2d(chan, 2*chan, 2, 2))
            chan = chan * 2
        self.middle_blks = nn.Sequential(*[NAFBlock(chan) for _ in range(middle_blk_num)])
        for num in dec_blk_nums:
            self.ups.append(nn.Sequential(nn.Conv2d(chan, chan * 2, 1, bias=False), nn.PixelShuffle(2)))
            chan = chan // 2
            self.decoders.append(nn.Sequential(*[NAFBlock(chan) for _ in range(num)]))

    def forward(self, x):
        inp = x
        x = self.intro(x)
        encs = []
        for encoder, down in zip(self.encoders, self.downs):
            x = encoder(x)
            encs.append(x)
            x = down(x)
        x = self.middle_blks(x)
        for decoder, up, enc_skip in zip(self.decoders, self.ups, encs[::-1]):
            x = up(x)
            x = x + enc_skip
            x = decoder(x)
        x = self.ending(x)
        return x + inp

print("✅ NAFNet architecture defined.")

# --- 2. Load Pre-trained Weights ---

# Check GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f">>> Running on device: {device}")

# Initialize Model (Width=32 configuration for GoPro)
model_naf = NAFNet(img_channel=3, width=32, middle_blk_num=1,
                   enc_blk_nums=[1, 1, 1, 28], dec_blk_nums=[1, 1, 1, 1])

# Download and Load Weights
weight_path = "NAFNet-GoPro-width32.pth"
weight_url = "https://huggingface.co/nyanko7/nafnet-models/resolve/main/NAFNet-GoPro-width32.pth"

if not os.path.exists(weight_path):
    print(">>> Downloading pre-trained weights from HuggingFace...")
    os.system(f"curl -L -o {weight_path} {weight_url}")

print(">>> Loading weights into model...")
try:
    checkpoint = torch.load(weight_path, map_location=device)
    param_dict = checkpoint['params'] if 'params' in checkpoint else checkpoint

    # strict=False allows us to load weights even if there are minor mismatches
    # (though here they should match perfectly)
    model_naf.load_state_dict(param_dict, strict=False)
    model_naf.to(device)
    print("✅ Model loaded successfully and moved to GPU.")
except Exception as e:
    print(f"❌ Error loading weights: {e}")

✅ NAFNet architecture defined.
>>> Running on device: cuda
>>> Downloading pre-trained weights from HuggingFace...
>>> Loading weights into model...
✅ Model loaded successfully and moved to GPU.


In [ ]:
# Cell 3.3: Fine-tuning (Train Split: First 80 Images)
# ====================================================
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import cv2
import numpy as np
import random
from tqdm import tqdm
import torch.nn as nn
import os

# 1. Define Dataset with explicit splitting
class TrafficTrainDataset(Dataset):
    def __init__(self, gt_dir, blur_dir, patch_size=256):
        self.gt_dir = gt_dir
        self.blur_dir = blur_dir
        self.patch_size = patch_size

        # Sort files to ensure deterministic order
        all_files = sorted(os.listdir(gt_dir))

        # Select ONLY the first 80 images for training
        self.files = all_files[:80]
        print(f">>> Dataset [TRAIN]: Loaded {len(self.files)} images (traffic_0000 - traffic_0079).")

    def __len__(self): return len(self.files)

    def __getitem__(self, idx):
        name = self.files[idx]

        # Load images
        img_gt = cv2.cvtColor(cv2.imread(os.path.join(self.gt_dir, name)), cv2.COLOR_BGR2RGB)
        img_blur = cv2.cvtColor(cv2.imread(os.path.join(self.blur_dir, name)), cv2.COLOR_BGR2RGB)

        # Normalize
        img_gt = img_gt.astype(np.float32) / 255.0
        img_blur = img_blur.astype(np.float32) / 255.0

        # Random Crop (Data Augmentation for training)
        h, w, c = img_gt.shape
        if h > self.patch_size and w > self.patch_size:
            rnd_h = random.randint(0, h - self.patch_size)
            rnd_w = random.randint(0, w - self.patch_size)
            img_gt = img_gt[rnd_h:rnd_h+self.patch_size, rnd_w:rnd_w+self.patch_size, :]
            img_blur = img_blur[rnd_h:rnd_h+self.patch_size, rnd_w:rnd_w+self.patch_size, :]
        else:
            img_gt = cv2.resize(img_gt, (self.patch_size, self.patch_size))
            img_blur = cv2.resize(img_blur, (self.patch_size, self.patch_size))

        return (torch.from_numpy(img_blur.transpose(2,0,1)),
                torch.from_numpy(img_gt.transpose(2,0,1)))

# 2. Setup DataLoader
# Use DIRS from your existing environment setup
train_ds = TrafficTrainDataset(DIRS["GT"], DIRS["BLUR"])
train_loader = DataLoader(train_ds, batch_size=4, shuffle=True, num_workers=2)

# Optimizer & Loss
optimizer = optim.AdamW(model_naf.parameters(), lr=1e-4, betas=(0.9, 0.9)) # Lower learning rate for fine-tuning
criterion = nn.L1Loss()

# 3. Training Loop
EPOCHS = 5
print(f"\n>>> Starting Fine-tuning for {EPOCHS} epochs...")

model_naf.train()

for epoch in range(EPOCHS):
    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}")
    for blur, gt in loop:
        blur = blur.to(device)
        gt = gt.to(device)

        # Forward pass
        optimizer.zero_grad()
        output = model_naf(blur)
        loss = criterion(output, gt)

        # Backward pass
        loss.backward()

        # Gradient Clipping
        torch.nn.utils.clip_grad_norm_(model_naf.parameters(), 0.01)
        optimizer.step()

        loop.set_postfix(loss=f"{loss.item():.5f}")

print("✅ Cell 3.3: Fine-tuning complete. Model weights updated.")

>>> Dataset [TRAIN]: Loaded 80 images (traffic_0000 - traffic_0079).

>>> Starting Fine-tuning for 5 epochs...


Epoch 5/5: 100%|██████████| 20/20 [00:07<00:00,  2.56it/s, loss=0.01002]

✅ Cell 3.3: Fine-tuning complete. Model weights updated.


In [ ]:
# Cell 3.4: Evaluation with Smart Metric Selection (PSNR/SSIM if GT exists, NIQE otherwise)
# =========================================================================================
import pandas as pd
from skimage import metrics
import cv2
import numpy as np
from tqdm import tqdm
import os
import torch
import pyiqa  # <--- NEW: Import PyIQA

# 1. Initialize NIQE Metric
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(">>> Loading NIQE metric...")
try:
    niqe_metric = pyiqa.create_metric('niqe', device=device)
except Exception as e:
    print(f"⚠️ Warning: Could not load NIQE: {e}")
    niqe_metric = None

model_naf.eval()
results_naf = []

# --- Split Logic: Updated to include ALL remaining images ---
# 0-79: Used for Training
# 80-99: Synthetic Validation (Has GT)
# 100+: Real World Images (No GT)
all_files = sorted(os.listdir(DIRS["BLUR"]))
test_files = all_files[80:] # <--- NEW: Select EVERYTHING after the training set

print(f">>> Evaluation Set: {len(test_files)} images (traffic_0080 onwards).")
print(f">>> Saving results to: {DIRS['NAF_FT_OUT']}")

for f in tqdm(test_files, desc="NAFNet Inference"):
    # Paths
    path_blur = os.path.join(DIRS["BLUR"], f)
    path_gt = os.path.join(DIRS["GT"], f)
    path_out = os.path.join(DIRS["NAF_FT_OUT"], f)

    # Check if GT exists
    has_gt = os.path.exists(path_gt)

    # Read Blur Image
    img_blur_orig = cv2.imread(path_blur)
    if img_blur_orig is None: continue # Skip if file is corrupt
    img_blur_orig = cv2.cvtColor(img_blur_orig, cv2.COLOR_BGR2RGB)

    # Preprocess
    img_tensor = torch.from_numpy(img_blur_orig.astype(np.float32)/255.0)
    img_tensor = img_tensor.permute(2,0,1).unsqueeze(0).to(device)

    # Inference
    with torch.no_grad():
        # Handle padding (Network requires multiple of 32)
        _, _, h, w = img_tensor.shape
        h_n = (h // 32) * 32
        w_n = (w // 32) * 32
        inp = img_tensor[:, :, :h_n, :w_n]

        output = model_naf(inp)

    # Post-process
    out_img = output.squeeze().cpu().permute(1,2,0).numpy()
    out_img = np.clip(out_img, 0, 1)
    out_uint8 = (out_img * 255.0).astype(np.uint8)

    # Save Image to the new folder
    cv2.imwrite(path_out, cv2.cvtColor(out_uint8, cv2.COLOR_RGB2BGR))

    # --- Metrics Calculation Logic ---
    row = {"Image": f, "Method": "NAFNet_FineTuned", "Has_GT": has_gt}

    # 1. Calculate NIQE (Applicable for ALL images)
    if niqe_metric:
        # NIQE expects tensor input (NCHW)
        out_tensor = torch.from_numpy(out_uint8).permute(2,0,1).unsqueeze(0).float() / 255.0
        row["NIQE"] = niqe_metric(out_tensor.to(device)).item()
    else:
        row["NIQE"] = None

    # 2. Calculate PSNR/SSIM (Only if GT exists)
    if has_gt:
        img_gt = cv2.imread(path_gt)
        if img_gt is not None:
            img_gt = cv2.cvtColor(img_gt, cv2.COLOR_BGR2RGB)
            # Crop GT to match output size (due to padding)
            gt_crop = img_gt[:h_n, :w_n, :]

            row["PSNR"] = metrics.peak_signal_noise_ratio(gt_crop, out_uint8)
            row["SSIM"] = metrics.structural_similarity(gt_crop, out_uint8, channel_axis=2, win_size=3)
        else:
            row["PSNR"] = None
            row["SSIM"] = None
    else:
        # No GT available (Real-world images)
        row["PSNR"] = None
        row["SSIM"] = None

    results_naf.append(row)

# Save Metrics with a DISTINCT filename
csv_path = os.path.join(DIRS["METRICS"], "metrics_nafnet_finetuned_mixed.csv")
pd.DataFrame(results_naf).to_csv(csv_path, index=False)

print(f"\n✅ Cell 3.4 Complete!")
print(f"   - Images saved to: {DIRS['NAF_FT_OUT']}")
print(f"   - Metrics saved to: {csv_path}")

>>> Loading NIQE metric...
Downloading: "https://huggingface.co/chaofengc/IQA-PyTorch-Weights/resolve/main/niqe_modelparameters.mat" to /root/.cache/torch/hub/pyiqa/niqe_modelparameters.mat



100%|██████████| 8.15k/8.15k [00:00<00:00, 34.2MB/s]


>>> Evaluation Set: 30 images (traffic_0080 onwards).
>>> Saving results to: /content/drive/My Drive/ECE253_Project/5_Output_NAFNet_FineTuned


NAFNet Inference: 100%|██████████| 30/30 [00:26<00:00,  1.14it/s]



✅ Cell 3.4 Complete!
   - Images saved to: /content/drive/My Drive/ECE253_Project/5_Output_NAFNet_FineTuned
   - Metrics saved to: /content/drive/My Drive/ECE253_Project/Metrics/metrics_nafnet_finetuned_mixed.csv
